In [ ]:
import chromadb

CHROMA_PATH = '.chroma'
COLLECTION_NAME = 'shrub_of_life'


def open_db(path = CHROMA_PATH, collection_name = COLLECTION_NAME):
    """Open the ChromaDB and return the collection."""
    client = chromadb.PersistentClient(path = path)
    collection = client.get_collection(name = collection_name)
    return collection

def find_closest(collection, seq_id, n = 10):
    """Find the N closest sequences to a given ID.
    
    Args:
        collection: ChromaDB collection
        seq_id: Sequence ID to query
        n: Number of closest sequences to return (default 10)
    
    Returns:
        dict with 'ids', 'distances', and 'metadatas' for the N closest sequences
    """
    # Get the embedding for the query sequence
    result = collection.get(ids = [seq_id], include = ['embeddings'])
    if not result['ids']:
        raise ValueError(f'Sequence ID not found: {seq_id}')
    
    query_embedding = result['embeddings'][0]
    
    # Query for N+1 closest (includes the query itself)
    results = collection.query(
        query_embeddings = [query_embedding],
        n_results = n + 1,
        include = ['distances', 'metadatas']
    )
    
    # Remove the query sequence from results (distance 0)
    ids = results['ids'][0]
    distances = results['distances'][0]
    metadatas = results['metadatas'][0]
    
    filtered = [(i, d, m) for i, d, m in zip(ids, distances, metadatas) if i != seq_id]
    filtered = filtered[:n]  # Ensure we return exactly n results
    
    return {
        'ids': [x[0] for x in filtered],
        'distances': [x[1] for x in filtered],
        'metadatas': [x[2] for x in filtered]
    }

In [ ]:
# Open the database
collection = open_db()
print(f'Collection: {collection.name}')
print(f'Count: {collection.count():,} sequences')

Collection: shrub_of_life
Count: 4,776,770 sequences


In [6]:
# Example: find closest sequences to the first entry
# Get a sample ID first
sample = collection.peek(limit = 1)
sample_id = sample['ids'][0]
print(f'Query ID: {sample_id}')

# Find 10 closest
closest = find_closest(collection, sample_id, n = 10)

print(f'\nClosest sequences:')
for i, (seq_id, dist, meta) in enumerate(zip(closest['ids'], closest['distances'], closest['metadatas']), 1):
    print(f'{i:2}. {seq_id}  distance={dist:.4f}  length={meta["length"]}')

Query ID: SFE_1_S_c_10

Closest sequences:
 1. SFE_1_S_c_122567  distance=0.6251  length=5566
 2. SE_17_c_96112  distance=0.6299  length=6816
 3. SE_3_c_249659  distance=0.6394  length=6429
 4. SFE_2_S_c_151156  distance=0.6468  length=14979
 5. SE_19_c_168191  distance=0.6470  length=7566
 6. SE_17_c_542063  distance=0.6475  length=5917
 7. SE_4_c_146066  distance=0.6505  length=5900
 8. SFE_2_W_c_42953  distance=0.6525  length=6425
 9. SE_8_c_386671  distance=0.6539  length=6447
10. SE_1_c_111408  distance=0.6586  length=5371


In [9]:
# Find 10 closest
sample_id = 'SFE_1_S_c_43029'
closest = find_closest(collection, sample_id, n = 20)

print(f'\nClosest sequences:')
for i, (seq_id, dist, meta) in enumerate(zip(closest['ids'], closest['distances'], closest['metadatas']), 1):
    print(f'{i:2}. {seq_id}  distance={dist:.4f}  length={meta["length"]}')


Closest sequences:
 1. SFE_2_S_c_106004  distance=0.0254  length=237764
 2. SFE_7_S_c_178807  distance=0.1195  length=246514
 3. SFE_7_S_c_83643  distance=0.1204  length=245127
 4. SFE_1_S_c_25667  distance=0.1273  length=241643
 5. SFE_1_S_c_34111  distance=0.1335  length=258008
 6. SFE_7_S_c_90700  distance=0.1336  length=238638
 7. SFE_1_S_c_21893  distance=0.1352  length=153779
 8. SFE_1_S_c_56398  distance=0.1381  length=139427
 9. SFE_6_S_c_109514  distance=0.1425  length=120982
10. SFE_1_S_c_43555  distance=0.1436  length=186974
11. SFE_7_S_c_144437  distance=0.1439  length=212559
12. SFE_7_S_c_207986  distance=0.1443  length=235184
13. SFE_1_S_c_66072  distance=0.1459  length=175127
14. SFE_2_S_c_140229  distance=0.1460  length=220875
15. SFE_1_S_c_49670  distance=0.1462  length=219812
16. SFE_1_S_c_34088  distance=0.1487  length=247438
17. SFE_8_S_c_34055  distance=0.1493  length=269816
18. SFE_7_S_c_95149  distance=0.1507  length=197156
19. SFE_5_S_c_79247  distance=0.1510  

In [ ]:
# Find 10 closest
sample_id = 'SFE_1_S_c_16172'
closest = find_closest(collection, sample_id, n = 10)

print(f'\nClosest sequences:')
for i, (seq_id, dist, meta) in enumerate(zip(closest['ids'], closest['distances'], closest['metadatas']), 1):
    print(f'{i:2}. {seq_id}  distance={dist:.4f}  length={meta["length"]}')


Closest sequences:
 1. SFE_4_W_c_94076  distance=0.0718  length=455972
 2. SFE_1_S_c_32453  distance=0.0722  length=2231819
 3. SFE_2_S_c_53619  distance=0.0727  length=1434939
 4. SFE_2_S_c_12943  distance=0.0815  length=721377
 5. SFE_6_W_c_29254  distance=0.0820  length=360510
 6. SFE_4_W_c_64917  distance=0.0905  length=536389
 7. SE_7_c_126886  distance=0.0907  length=805138
 8. SE_10_c_21282  distance=0.0938  length=1337801
 9. SFE_2_S_c_23093  distance=0.0977  length=364243
10. SFE_2_W_c_145484  distance=0.1039  length=211968
